# Classes
## Item 22: Prefer Helper Classes over Bookkeeping with Dictionaries and Tuples

- Avoid making dictionaries with values that are other dictionaries or long tuples.

- Use namedtuple for lightweight, immutable data containers before you need the flexibility of a full class.

- Move your bookkeeping code to use multiple helper classes when your internal state dictionaries get complicated.

In [1]:

import collections
Grade = collections.namedtuple('Grade', ('score', 'weight')) # a record of data


# Example 12
class Subject(object):
    def __init__(self):
        self._grades = []

    def report_grade(self, score, weight):
        self._grades.append(Grade(score, weight))

    def average_grade(self):
        total, total_weight = 0, 0
        for grade in self._grades:
            total += grade.score * grade.weight
            total_weight += grade.weight
        return total / total_weight


# Example 13
class Student(object):
    def __init__(self):
        self._subjects = {}

    def subject(self, name):
        if name not in self._subjects:
            self._subjects[name] = Subject()
        return self._subjects[name]

    def average_grade(self):
        total, count = 0, 0
        for subject in self._subjects.values():
            total += subject.average_grade()
            count += 1
        return total / count


# Example 14
class Gradebook(object):
    def __init__(self):
        self._students = {}

    def student(self, name):
        if name not in self._students:
            self._students[name] = Student()
        return self._students[name]


# Example 15
book = Gradebook()
albert = book.student('Albert Einstein')
math = albert.subject('Math')
math.report_grade(80, 0.10)
math.report_grade(80, 0.10)
math.report_grade(70, 0.80)
gym = albert.subject('Gym')
gym.report_grade(100, 0.40)
gym.report_grade(85, 0.60)
print(albert.average_grade())

81.5


## Item 23: Accept Functions for Simple Interfaces Instead of Classes

- Instead of defining and instantiating classes, functions are often all you need for simple interfaces between components in Python.

- References to functions and methods in Python are first class, meaning they can be used in expressions like any other type.

- The __call__ special method enables instances of a class to be called like plain Python functions.

- When you need a function to maintain state, consider defining a class that provides the __call__ method instead of defining a stateful closure (see Item 15: “Know How Closures Interact with Variable Scope”).

In [3]:
from collections import defaultdict

def log_missing():
    print('Key added')
    return 0

current = {'green': 12, 'blue': 3}
increments = [
    ('red', 5),
    ('blue', 17),
    ('orange', 9),
]
result = defaultdict(log_missing, current)
print('Before:', dict(result))
for key, amount in increments:
    result[key] += amount
print('After: ', dict(result))

Before: {'green': 12, 'blue': 3}
Key added
Key added
After:  {'green': 12, 'blue': 20, 'red': 5, 'orange': 9}


In [7]:
class BetterCountMissing(object):
    def __init__(self):
        self.added = 0

    def __call__(self):
        self.added += 1
        return self.added

counter = BetterCountMissing()
counter()

1

In [8]:
counter()

2

## Item 24: Use @classmethod Polymorphism to Construct Object Generically
- Python only supports a single constructor per class, the __init__ method. 
- Use @classmethod to define alternative constructors for your classes.
- Use class method polymorphism to provide generic ways to build and connect concrete subclasses.

In [20]:
from random import randint

class Cheese(object):
    def __init__(self, num_holes=0):
        "defaults to a solid cheese"
        self.number_of_holes = num_holes
        
    def __str__(self):
        return str(self.number_of_holes)

    @classmethod
    def random(cls): # act as multiple constructors
        return cls(randint(0, 100))

    @classmethod 
    def slightly_holey(cls): # act as multiple constructors
        return cls(randint(0,33))

    @classmethod
    def very_holey(cls): # act as multiple constructors
        return cls(randint(66, 100))

In [22]:
gouda = Cheese(10)
print(gouda)

10


In [23]:
emmentaler = Cheese.random()
print(emmentaler)

56


In [25]:
leerdammer = Cheese.slightly_holey()
print(leerdammer)

1


In [26]:
test_very_holey = Cheese.very_holey()
print(test_very_holey)

94


## Item 25: Initialize Parent Classes with super

- Python’s standard method resolution order (MRO) solves the problems of superclass initialization order and diamond inheritance.

- Always use the super built-in function to initialize parent classes.

In [27]:
class MyBaseClass(object):
    def __init__(self, value):
        self.value = value
        
class Implicit(MyBaseClass):
    def __init__(self, value):
        super().__init__(value * 2)
        
print(Implicit(10).value)

20


## Item 26: Use Multiple Inheritance Only for Mix-in Utility Classes
- Avoid using multiple inheritance if mix-in classes can achieve the same outcome.

## Item 27: Prefer Public Attributes Over Private Ones

- Private attributes aren’t rigorously enforced by the Python compiler.

- Plan from the beginning to allow subclasses to do more with your internal APIs and attributes instead of locking them out by default.

- Use documentation of protected fields to guide subclasses instead of trying to force access control with private attributes.

- Only consider using private attributes to avoid naming conflicts with subclasses that are out of your control.

In [55]:
class MyObject(object):
    def __init__(self):
        self.public_field = 5
        self._protected_field = 10
        self.__private_field = 20

    def get_private_field(self):
        return self.__private_field
    
    def _procted_method(self):
        print("call procted method")
    
    def __private_method(self):
        print("Call private method")

foo = MyObject()
foo.public_field

5

In [48]:
foo.get_private_field()

20

In [52]:
foo._protected_field

10

In [50]:
foo.__private_field

AttributeError: 'MyObject' object has no attribute '__private_field'

In [54]:
foo._procted_method()

call procted method


In [56]:
foo.__private_method()

AttributeError: 'MyObject' object has no attribute '__private_method'

In [59]:
foo._MyObject__private_field = 10
foo.get_private_field()

10

In [60]:
print(foo.__dict__)

{'public_field': 5, '_protected_field': 10, '_MyObject__private_field': 10}


## Item 28: Inherit from collections.abc for Custom Container Types
- Inherit directly from Python’s container types (like list or dict) for simple use cases.

- Beware of the large number of methods required to implement custom container types correctly.

- Have your custom container types inherit from the interfaces defined in *collections.abc* to ensure that your classes match required interfaces and behaviors.

In [63]:
class ListBasedSet(collections.abc.Set):
    ''' Alternate set implementation favoring space over speed
        and not requiring the set elements to be hashable. '''
    def __init__(self, iterable):
        self.elements = lst = []
        for value in iterable:
            if value not in lst:
                lst.append(value)

    def __str__(self):
        return str(self.elements)
    
    def __iter__(self):
        return iter(self.elements)

    def __contains__(self, value):
        return value in self.elements

    def __len__(self):
        return len(self.elements)

s1 = ListBasedSet('abcdef')
s2 = ListBasedSet('defghi')
overlap = s1 & s2            # The __and__() method is supported automatically
print(overlap)

['d', 'e', 'f']
